In [25]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt, pi, exp

#from pandas_datareader import data
from argon2 import Parameters
from statsmodels.tsa import stattools
from statsmodels.tsa import arima_model
from statsmodels.graphics import gofplots
from statsmodels.regression import linear_model

from scipy import stats
import scipy.integrate
import scipy.special
import yfinance as yf
from sklearn.linear_model import LinearRegression


import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
from scipy.optimize import minimize


In [49]:
i = complex(0, 1)
# To be used in the Heston pricer
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s
    # Calculate d
    d1 = (prod-kappa) ** 2
    d2 = (sigma**2) * (-2 * i * s + s**2)
    d = np.sqrt(d1 + d2)
    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2
    # Calculate first exponential
    exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, -2 * theta * kappa/ (sigma**2))
    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))
    return mainExp1 * mainExp2

In [71]:
# Heston Pricer
def priceHestonMid(St, K, r, q, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations
    element1 = 0.5 * (St * np.exp(-q * T) - K * np.exp(-r * T))
    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        numerator1 = fHeston(s2, St, K, r, T,
                             sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T,
                               sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1
        P += ds * (numerator1 - numerator2) / denominator
    element2 = P / np.pi
    return np.real((element1 + element2))

In [73]:
# Test Parameters

S_0_CSCO = 43.32            # stock price: CSCO
S_0_GE = 65.29              # stock price: GE
r = 1.53/100                # interest rate
q_csco = 0.0152             # dividend yield CSCO
q_ge = 0.0032               # dividend yield GE
K_CSCO = 55                 # specific strike CSCO
K_GE = 10                   # specific strike GE
T = 150/250                 # 30 weeks: 150 trading days
marketPrices_tab_CSCO = 0.54


In [74]:
x0_CSCO = [0.2, 0.2, 0.2, 0.2, -0.2]
obj_CSCO = lambda alpha : (0.54 - priceHestonMid(S_0_CSCO, K_CSCO, r, q_csco, T, alpha[0], alpha[1], alpha[2], alpha[3], alpha[4]))**2
opt = scipy.optimize.minimize(obj_CSCO, x0_CSCO, options={'tol': 1e-6, 'maxiter':1e6})
print(opt.x)

/Users/z.scialom/Documents/IIT_courses_SPRING/MATH584_Algo_Trading/PyCharm_projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: OptimizeWarning: Unknown solver options: tol
  This is separate from the ipykernel package so we can avoid doing imports until


[ 0.20151791  0.19629905  0.19688137  0.15033888 -0.20281649]


In [81]:
def putPriceHestonMid(St, K, r, q, T, call_price):
    return call_price + K * np/exp(-r * T) - St * np/exp(-q * T)
    #return put_price

In [92]:
x0_GE = [0.2, 0.2, 0.2, 0.2, 0.2]
obj_GE = lambda alpha : (0.87 - priceHestonMid(S_0_GE, K_GE, r, q_ge, T, alpha[0], alpha[1], alpha[2], alpha[3], alpha[4]) - K_GE * np.exp(-r * T) + S_0_GE * np.exp(-q_ge * T))**2

opt = scipy.optimize.minimize(obj_GE, x0_GE, options={'tol': 1e-6, 'maxiter':1e6})
print(opt.x)

/Users/z.scialom/Documents/IIT_courses_SPRING/MATH584_Algo_Trading/PyCharm_projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: OptimizeWarning: Unknown solver options: tol
  after removing the cwd from sys.path.
/Users/z.scialom/Documents/IIT_courses_SPRING/MATH584_Algo_Trading/PyCharm_projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/Users/z.scialom/Documents/IIT_courses_SPRING/MATH584_Algo_Trading/PyCharm_projects/venv/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/z.scialom/Documents/IIT_courses_SPRING/MATH584_Algo_Trading/PyCharm_projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  


[0.19890496 0.20070852 0.1969149  0.14935308 0.19929772]
